In [1]:
# Bot3.ipynb
# the bot, mugged...
#  Supports changing *date* to the day of week
#
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
import pickle
intents = json.loads(open('intents3.json').read())
tags = []
pats = []
for i in intents['intents']:
    # print(i['tag'])
    tags.append(i['tag'])
    pats.append(i['patterns'])
print("Tags: ")
print(tags)
print("Pats")
print(pats)
# print("intents: ", intents)
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

2021-09-02 10:10:13.065345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-02 10:10:13.065400: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Tags: 
['date', 'greeting', 'goodbye', 'thanks', 'noanswer', 'options', 'quatch', 'doch', 'nein', 'ja', 'stimmt']
Pats
[['What is today?', 'Today is?'], ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], [], ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], ['What does quatsch mean?', 'Quatch means what?'], ['What does dock mean?', 'Doch means what?'], ['What does nein mean in German', 'Nein means what?'], ['What does ja mean?', 'Ja means what?'], ['What does stimmt mean?', 'Stimmt means what?']]


2021-09-02 10:10:14.468118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-02 10:10:14.468175: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-02 10:10:14.468220: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-964CGUG): /proc/driver/nvidia/version does not exist
2021-09-02 10:10:14.468563: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        if show_details:  # bob was here
            print("word: %s" % s)
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    # print("ints: ", ints)
    tag = ints[0]['intent']
    # print("tag: ", tag)
    # print("intents: ", intents)
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if (i['tag']== tag):
            result = random.choice(i['responses'])
            break
        c_list = i['context']
        # print("cList[0]: ", c_list[0])        
        if len(c_list[0]) > 0:
            print("Current context: ", i['context'])
    # print("result: ", result)        
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [3]:
#Creating Text GUI
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np
from datetime import date
wordy = False
#
while True:
    if wordy == True:
        print("Tags: ")
        print(tags)
        print("Pats: ")
        print(pats)
    msg = input("You: ")
    if msg == "exit":
        break;
    res = chatbot_response(msg)
    # parse for *date* and replace with date.today()
    # DEBUG, off
    # today = date.today().strftime("%A")  #("%m/%d/%Y")
    # res = res.replace("*date*", today)
    print(">>> Bot: ", res)
print(">>> Done!")
#        res = chatbot_response(msg)
#        ChatLog.insert(END, "Bot: " + res + '\n\n')


You: what does quatsch mean?


2021-09-02 10:10:28.360046: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-02 10:10:28.360897: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1992000000 Hz


>>> Bot:  Quatsch translates to nonsense.
You: What is nein?
>>> Bot:  Nein translates to no.
You: what is ja?
>>> Bot:  Ja means yes.
You: hello
>>> Bot:  Hello, thanks for asking
You: greetings
>>> Bot:  Good to see you again
You: what do you do?
>>> Bot:  I can guide you through the meaning of a few German words. 
You: exit
>>> Done!
